# Lab 2: Distance and K-meand Clustering

<a target="_blank" href="https://raw.githubusercontent.com/drchadvidden/courseMaterials/refs/heads/main/UnsupervisedLearning/Labs/Lab%202/Lab_2.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Lab Instructions

Run each of the coding cells. For tutorial example cells, understand the commands and check that the outputs make sense. For exercise cells, write your own code where indicated to generate the correct output. Give text explanations where indicated.

### Submission:
Complete the following notebook in order. Once done, save the notebook, print the file as a .pdf, and upload the resulting file to the Canvas course assignment.

### Rubric:
15 total points, 5 points to running tutorial example cells and saving outputs, 10 points for completing exercises.

### Deadline:
Tuesday at midnight after the lab is assigned.

# Tutorial: Distance and the curse of dimensionality

## Distance Metrics

Let  
$$
\mathbf{x} = (x_1, \dots, x_d), \quad \mathbf{y} = (y_1, \dots, y_d)
$$
be two vectors (data points) in $\mathbb{R}^d$. Recall our three common distance metrics.

Euclidean Distance ($\ell_2$)
$$
d_2(\mathbf{x}, \mathbf{y}) = \sqrt{\sum_{i=1}^d (x_i - y_i)^2}
$$
This is the “straight-line” distance and the default choice in many algorithms (e.g. k-means).

Manhattan Distance ($\ell_1$)
$$
d_1(\mathbf{x}, \mathbf{y}) = \sum_{i=1}^d |x_i - y_i|
$$
Often used when movement is constrained along axes or when robustness to outliers is desired.

Supremum Distance ($\ell_{\infty}$)
$$
d_\infty(\mathbf{x}, \mathbf{y}) = \max_{1 \leq i \leq d} |x_i - y_i |
$$
Measures the largest coordinate difference only, and ignores other differences.

In [ ]:
import numpy as np

# this example code calculates distance 3 ways
x = np.array([2, 1, 5])
y = np.array([6, 4, 3])

euclidean = np.linalg.norm(x - y, ord=2)
manhattan = np.linalg.norm(x - y, ord=1)
chebyshev = np.linalg.norm(x - y, ord=np.inf)

print(f"Euclidean distance:  {euclidean:.3f}")
print(f"Manhattan distance: {manhattan:.3f}")
print(f"Chebyshev distance: {chebyshev:.3f}")

## The Curse of Dimensionality

As the number of dimensions increases, distances between points become less informative.

Intuition:
*   In high dimensions, points tend to be far apart
*   The distance to the nearest neighbor becomes almost the same as the distance to the farthest neighbor
*   This degrades methods based on distance (k-NN, k-means, clustering) because nearness and farness is blurry

To demonstrate, we generate random points in the unit hypercube $[0,1]^d$ ($d$-dimensional cube) and examine distances from a fixed reference point. As
$d$ increases, the difference between the minimum and maximum distance shrinks relative to their size. Points become almost equally distant.


In [ ]:
import numpy as np

def distance_stats(d, n=1000):
    x0 = np.random.rand(d) # random reference vector
    X = np.random.rand(n, d) # n random vectors to compare to reference vector
    dists = np.linalg.norm(X - x0, axis=1) #l_2 (Euclidean) distance
    return dists.min(), dists.max(), dists.mean(), x0, X

dimensions = [2, 5, 10, 20, 50, 100, 500, 1000, 5000, 10000]

# small dimension case
dmin, dmax, dmean, x0, X = distance_stats(2)
print("Example in small dimension:")
print(x0)
print(X)
print(f"d={d:3d} | min={dmin:.3f}, max={dmax:.3f}, mean={dmean:.3f}")

# summarize many dimension findings
print("")
print("Example, summary stats as dimension increases")
for d in dimensions:
    dmin, dmax, dmean, x0, X = distance_stats(d)
    print(f"d={d:3d} | min={dmin:.3f}, max={dmax:.3f}, mean={dmean:.3f}")


# Tutorial: K-means

## Synthetic movie rating data

The k-means clustering algorithm represents each cluster by its corresponding cluster centroid. The algorithm would partition the input data into k disjoint clusters by iteratively applying the following two steps:

Form k clusters by assigning each instance to its nearest centroid.
Recompute the centroid of each cluster.
In this section, we perform k-means clustering on a toy example of movie ratings dataset. We first create the dataset as follows.

In [ ]:
import pandas as pd

ratings = [
    ['john',   5, 5, 2, 1],
    ['mary',   4, 5, 3, 2],
    ['bob',    4, 4, 4, 3],
    ['lisa',   2, 2, 4, 5],
    ['lee',    1, 2, 3, 4],
    ['harry',  2, 1, 5, 5],
    ['alex',   5, 4, 2, 2],
    ['emma',   4, 5, 2, 1],
    ['chris',  5, 4, 3, 2],
    ['nina',   4, 4, 3, 2],
    ['paul',   2, 2, 5, 4],
    ['sara',   1, 2, 4, 5],
    ['mike',   2, 1, 4, 4],
    ['lily',   2, 2, 5, 5],
    ['drew',   3, 3, 3, 3],
    ['jordan', 3, 4, 4, 3],
]

titles = ['user', 'Jaws', 'Star Wars', 'Exorcist', 'Saw']
movie_ratings = pd.DataFrame(ratings, columns=titles)
movie_ratings


Read through this dataset and see if you can identify some preference patters of the users. Our goal is to apply k-means clustering on the users to identify groups of users with similar movie preferences.

## K-means clustering (k=2)

The example below shows how to apply k-means clustering (with k=2) on the movie ratings data. We must remove the "user" column first before applying the clustering algorithm. Feature scaling is not needed here because all ratings are on the same 1-5 scale. The cluster assignment for each user is displayed as a dataframe object.

In [ ]:
from sklearn import cluster

data = movie_ratings.drop('user',axis=1)
k_means = cluster.KMeans(n_clusters=2, random_state=42, n_init=10)
k_means.fit(data)
labels = k_means.labels_
movie_ratings['Cluster ID'] = k_means.labels_
pd.DataFrame(labels, index=movie_ratings.user, columns=['Cluster ID'])

## Cluster centroids and naming

Reading the table, we see the k-means clustering algorithm assigns the first three users to cluster 1 (action movie fans) and the next three users to cluster 0 (horror movie fans), and so on. The results are consistent with our expectation. We can also display the centroid for each of the two clusters. Not these centriods are cluster centers, not actual users.

In [ ]:
centroids = k_means.cluster_centers_
pd.DataFrame(centroids,columns=data.columns)

Observe that cluster 0 has higher ratings for the horror movies whereas cluster 1 has higher ratings for action movies. It might be useful to rename these clusters in light of these observations.

In [ ]:
cluster_names = {
    0: 'Horror Fans',
    1: 'Action Fans'
}
movie_ratings['Cluster Name'] = movie_ratings['Cluster ID'].map(cluster_names)
movie_ratings


Cluster assignments can visually inspected. Feel free to check other movie comparisons.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select two movies for the x and y axes
x_col = 'Jaws'
y_col = 'Exorcist'

# Map cluster names to colors
colors = {'Horror Fans': 'red', 'Action Fans': 'blue'}
user_colors = movie_ratings['Cluster Name'].map(colors)

# Plot users
plt.scatter(movie_ratings[x_col], movie_ratings[y_col],
            c=user_colors, s=100, alpha=0.6, edgecolor='k')

# Plot centroids
centroids_xy = k_means.cluster_centers_[:, [movie_ratings.columns.get_loc(x_col)-1,
                                            movie_ratings.columns.get_loc(y_col)-1]]
plt.scatter(centroids_xy[:,0], centroids_xy[:,1],
            c=['red','blue'], s=300, marker='X', edgecolor='k', linewidth=2)

# Add legend
for name, color in colors.items():
    plt.scatter([], [], c=color, label=name)
plt.legend()

plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title(f'Movie Ratings Clusters: {x_col} vs {y_col}')
plt.grid(True)
plt.show()

## Cluster assignment for new data

The cluster centroids can be applied to other users to determine their cluster assignments by simply finding the nearest centroid.

In [ ]:
import numpy as np

testData = np.array([[4,5,1,2],[3,2,4,4],[2,3,4,1],[3,2,3,3],[5,4,1,4]])
labels = k_means.predict(testData)
labels = labels.reshape(-1,1)
usernames = np.array(['paul','kim','liz','tom','bill']).reshape(-1,1)
cols = movie_ratings.columns.tolist()
newusers = pd.DataFrame(
    data = np.column_stack((usernames, testData, labels)),
    columns = ['user', 'Jaws', 'Star Wars', 'Exorcist', 'Saw', 'Cluster ID']
)
newusers

## Elbow method for cluster identification

To determine the number of clusters in the data, we can apply k-means with varying number of clusters from 1 to 6 and compute their corresponding within-cluster sum-of-squared (WCSS) errors as shown in the example below. The "elbow" in the plot of SSE versus number of clusters can be used to estimate the number of clusters. 2 clusters has a strong "elbow" indicating this is likely best, though maybe a 3 cluster solution is worth exploring.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

numClusters = [1,2,3,4,5,6,7,8]
SSE = []
for k in numClusters:
    k_means = cluster.KMeans(n_clusters=k, random_state=42, n_init=10)
    k_means.fit(data)
    SSE.append(k_means.inertia_)

plt.plot(numClusters, SSE)
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Select the two columns
X = df[["Age", "Spending Score (1-100)"]]

# Fit k-means (choose k=3 as a reasonable default)
kmeans = KMeans(n_clusters=4, random_state=42)
df["Cluster"] = kmeans.fit_predict(X)

# Plot
plt.figure()
plt.scatter(
    df["Age"],
    df["Spending Score (1-100)"],
    c=df["Cluster"]
)

plt.xlabel("Age")
plt.ylabel("Spending Score (1-100)")
plt.title("Age vs Spending Score with K-Means Clusters")

plt.show()


# Exercise(s): Curse of dimensionality and k-means





## Exercise 1: Comparing Distance Metrics

### Tasks:

1. Generate random points in  $\mathbb{R}^{10}$.

2. Compute the following distances:
   - Euclidean distance
   - Manhattan distance
   - Supremum distance

3. Repeat for $\mathbb{R}^{100}$. Organize your results in nice table or visual.

4. Compare how the distances change as the dimension increases. Which metric grows fastest with dimension? Which metric seems least sensitive to dimension?

In [ ]:
# Write your code for the exercise 1 here!

### Explain your findings here:




## Exercise 2: Nearest vs Farthest Neighbor

### Tasks

1. Fix a random point $x_0 \in [0,1]^d$

2. Generate 1,000 random points in the same space.

3. Compute all Euclidean distances to \( x_0 \).

4. Calculate the ratio $\frac{\max(\text{distance}) - \min(\text{distance})}{\min(\text{distance})}$

5. Repeat for $d = 2,\ 10,\ 50,\ 100...$. Organize your results in a nice table or visual. What happens to this ratio as the dimension increases? Why is this problematic for distance-based learning methods?


In [ ]:
# Write your code for the exercise 2 here!

### Explain your findings here:


## Exercise 3: K-means on movie rating

Use the above approach to find the $k=3$ clustering assignments.

### Tasks:

1. Assign cluster IDs to each user.

2. Inspect the centroids and give them suitable names. Find the size of each cluster and add to the centroid descriptive table.

3. Visualize your findings in 2D for pair(s) of movies.

4. Find the average distance of users to their cluster centroid for each cluster. Compare findings for each cluster.

5. Find the pairwise distance between each cluster.

### Explain your findings here:


## Exercise 4: K-means on customer segmentation

Explore a clustering of customer data.

### Tasks:

1. Process your data to decide which columns should be used (numeric only) and if normalization is needed.

2. Create a WCSS elbow plot for many $k$-means clusterings and identify a "best" cluster number $k$ to explore.

3. Assign cluster IDs to each user.

4. Inspect the centroids and give them suitable names. Find the size of each cluster and add to the centroid descriptive table.

5. Visualize your findings in 2D for pairs of customer attributes.

6. Revert your normalization and summarize the centroids in terms of the original data scale.

7. Analyze your clusters according to Gender. Do you see any relationships? Why does it make sense to not include the Gender feature when running $k$-means to create clusters in the first place?

8. (Bonus) Analyze a second clustering solution and indicated in your elbow plot and compare to 3-6 results above.

In [ ]:
import pandas as pd

# this file is also hosted on Kaggle: https://www.kaggle.com/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python
url = 'https://gist.githubusercontent.com/pravalliyaram/5c05f43d2351249927b8a3f3cc3e5ecf/raw/8bd6144a87988213693754baaa13fb204933282d/Mall_Customers.csv'
df = pd.read_csv(url)

print(df.head())
print(df.info())
print(df.describe())
print(df["Gender"].value_counts())

### Explain your findings here:


## Exercise 5: K-means exploration

Summarize the ideas of this example in the Scikit-learn documentation: https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py

### Explain your findings here:
